# Neural Lens: Image Captioning with InceptionV3 & LSTM

An end-to-end deep learning pipeline designed to bridge the gap between computer vision and natural language processing. This project automatically generates descriptive, human-like captions for any given image using the Flickr8k dataset.

## Key Features

- **Feature Extraction**: Utilizes InceptionV3 (pre-trained on ImageNet) to extract high-level visual features.

- **Sequence Modeling**: Implements a Deep LSTM decoder to generate text sequences word-by-word.

- **Search Strategies**: Features both Greedy Search and Beam Search (K-beams) for optimized sentence generation.

- **Quantitative Evaluation**: Measures performance using BLEU-1 and BLEU-2 scores with smoothing functions.

- **Custom Data Pipeline**: Efficiently handles large datasets using a Python generator and tf.data.Dataset to prevent memory overflow.

## 🛠️ Tech Stack

| Component               | Technology          |
| ----------------------- | ------------------- |
| Language                | Python 3            |
| Deep Learning           | TensorFlow / Keras  |
| Computer Vision         | InceptionV3         |
| NLP NLTK (BLEU metrics) | Keras Tokenizer     |
| Visualizations          | Matplotlib, Seaborn |

---

## 1. System and Utility Libraries

These modules handle file management, suppress noise in the output, and provide progress tracking.

* **os & warnings**: Used for directory navigation and filtering out unnecessary deprecation warnings.
* **re**: Regular expressions for text cleaning and preprocessing.
* **tqdm**: Provides progress bars for long-running tasks like image feature extraction.
* **Counter**: Used for vocabulary analysis and word frequency counts.

## 2. Data Handling and Visualization

Tools for managing numerical data and visualizing results.

* **numpy & pandas**: The core stack for array manipulation and managing metadata (like image-caption mappings).
* **matplotlib & seaborn**: Used to visualize training metrics and display images with their generated captions.
* **PIL (Pillow)**: The standard library for loading and resizing raw image files.

## 3. Deep Learning (TensorFlow/Keras)

The architecture follows a **CNN-RNN encoder-decoder** pattern:

### Image Processing (Encoder)

* **InceptionV3**: A pre-trained model used for **Transfer Learning** to extract feature vectors from images.
* **load_img & img_to_array**: Converts raw images into the format required by the neural network.

### Text Processing & Modeling (Decoder)

* **Tokenizer & pad_sequences**: Converts words into numeric sequences and ensures uniform input lengths.
* **LSTM & Embedding**: Layers designed to handle sequential data and capture the semantic relationship between words.
* **Dense, Dropout, BatchNormalization**: Layers used to regularize the model and map features to the final vocabulary size.

## 4. Evaluation Metrics

* **BLEU Score (NLTK)**: The primary metric used to evaluate the quality of the generated captions by comparing them against human-written references.

---

In [ ]:
import os 
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
import regex as re
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, add
from tensorflow.keras.layers import Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm_notebook
from collections import Counter

# Dataset Acquisition: Flickr8K

This cell automates the retrieval of the **Flickr8K dataset** directly from Kaggle. This dataset is a benchmark in the field of image captioning, consisting of 8,000 images each paired with five different human-annotated descriptions.

---

## 1. Automated Downloading

The code utilizes the kagglehub library to fetch the data programmatically, ensuring the environment is reproducible without manual file uploads.

* **kagglehub.dataset_download**: Downloads the specific adityajn105/flickr8k repository. This includes:
* **Images**: A folder containing the raw .jpg files.
* **Captions**: A text file (usually captions.txt) mapping image filenames to their respective descriptions.



## 2. Directory Verification

After the download completes, the script performs a health check on the file path:

* **path_to_dataset**: Stores the absolute local path where the files were saved.
* **os.listdir**: Iterates through the downloaded directory to confirm the presence of key files (e.g., the Images folder and the metadata files).

---

## Expected Directory Structure

Once successfully executed, you should see a file structure similar to this:

* Images/: 8,091 images in JPEG format.
* captions.txt: A CSV/text file containing the image IDs and their corresponding captions.


In [ ]:
# Import the flickr8k dataset from kaggle

import kagglehub
import os

# 1. Download the latest version
path_to_dataset = kagglehub.dataset_download("adityajn105/flickr8k")

print(f"Path to dataset files: {path_to_dataset}")

# 2. List the contents of the directory
files = os.listdir(path_to_dataset)

print("\nContents of the directory:")
for file in files:
    print(f"- {file}")

# Data Loading and Text Tokenization

This cell defines the initial functions for ingestion and text processing. It transforms the raw text file from the Flickr8K dataset into a format suitable for natural language modeling.

---

## 1. Path Configuration

* **images_directory**: Defines the source folder containing the .jpg files.
* **captions_path**: Targets the specific text file where image IDs are mapped to their descriptions.

## 2. Caption Ingestion (load_captions)

This function reads the metadata and performs basic text normalization:

* **Header Removal**: It slices the list [1:] to skip the CSV header (e.g., "image,caption").
* **Case Normalization**: Converts all text to **lowercase** using a list comprehension. This reduces the vocabulary size by ensuring "Dog" and "dog" are treated as the same token.

## 3. Text Vectorization (tokenize_captions)

This function prepares the text for the neural network using the Keras Tokenizer:

* **Vocabulary Building**: fit_on_texts analyzes the entire caption set to create an internal dictionary of every unique word.
* **Word-to-Index Mapping**: This step is crucial for converting human-readable sentences into sequences of integers that the **Embedding layer** of the model can process.

---

## 4. Execution and Validation

* The script calls load_captions to create the initial list of strings.
* **captions[:15:3]**: This slice prints every 3rd caption from the first 15 entries. This is a quick diagnostic check to verify that the text has been loaded correctly and the lowercase normalization was successful.

In [ ]:
images_directory = '/kaggle/input/flickr8k/Images/'
captions_path = '/kaggle/input/flickr8k/captions.txt'
def load_captions(file_path):
    with open(file_path, 'r') as f:
        captions = f.readlines()
        captions = [caption.lower() for caption in captions[1:]]
    return captions

def tokenize_captions(captions):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(captions)
    return tokenizer

captions = load_captions(captions_path)
captions[:15:3]

# Text Preprocessing and Cleaning

This cell implements a text cleaning pipeline to remove noise from the captions. Refining the text data is a critical step in reducing the model's vocabulary complexity and improving the accuracy of the generated descriptions.

---

## 1. The clean_text Function

This function applies a series of **Regular Expressions (Regex)** to standardize each caption:

- **Punctuation Removal**: re.sub(r'[^\w\s]', '', text) strips out all special characters and punctuation (e.g., commas, periods, exclamation marks).
- **Digit Removal**: re.sub(r'\d+', '', text) removes numbers, as they are often irrelevant to the general visual description of these images.
- **Whitespace Normalization**: re.sub(r'\s+', ' ', text).strip() replaces multiple spaces or tabs with a single space and trims leading/trailing whitespace.

## 2. Data Transformation

The cleaning process is applied to the raw dataset using a list comprehension:

- **Field Extraction**: caption.split(',')[1] separates the Image ID from the actual caption text (assuming a CSV-style format).
- **Batch Processing**: The clean_text function is called for every caption in the list, resulting in the cleaned_captions array.

---

## 3. Visualizing the Transformation

The cleaning process reduces "dirty" text into a "clean" format that the model can more easily learn.

| Input (Raw)                                        | Output (Cleaned)        |
| -------------------------------------------------- | ----------------------- |
| 1000268201_693b08cb0e.jpg,A child in a pink dress. | a child in a pink dress |
| 12345.jpg, Two dogs running!!                      | two dogs running        |

---

## 4. Verification

- **cleaned_captions[:15:2]**: This command prints every second cleaned caption from the first 15 entries. This allows for a manual inspection to ensure that punctuation and numbers have been successfully removed before the text is passed to the tokenizer.


In [ ]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

cleaned_captions = [clean_text(caption.split(',')[1]) for caption in captions]
cleaned_captions[:15:2]

# Sequence Tagging and ID Mapping

This cell prepares the final text format required for the **Encoder-Decoder** model architecture. It links the cleaned captions back to their respective image identifiers and adds structural markers for the sequence generation process.

---

## 1. Sequence Token Injection

The code adds two special "pseudo-words" to every caption:

- **start**: A "Start-of-Sequence" (SOS) token that signals to the model where to begin generating a description.
- **end**: An "End-of-Sequence" (EOS) token that tells the model when to stop generating words.

These tokens are essential for the inference phase, where the model must decide when a sentence is grammatically complete.

## 2. Image-to-Caption Mapping

The loop re-associates the text with its source image:

- **split(',')[0]**: Extracts the unique Image ID (e.g., 1000268201_693b08cb0e.jpg) from the original raw data.
- **Tab-Separated Formatting**: The ID and the tagged caption are joined using a \t (tab) delimiter.
- **Newline Addition**: A \n is added to ensure each entry is treated as a distinct line for future file exports or batch processing.

---

## 3. Data Structure Evolution

| Stage             | Data Format Example                |
| ----------------- | ---------------------------------- |
| **Raw**           | 123.jpg,A dog runs.                |
| **Cleaned**       | a dog runs                         |
| **Final Mapping** | 123.jpg \t start a dog runs end \n |

---

## 4. Verification and Dataset Scale

- **captions_IDs[:20:3]**: Samples every third entry from the first twenty to confirm the start, end, and tab-spacing are correctly applied.
- **len(captions_IDs)**: Returns the total count of captions in the dataset (expected to be approximately **40,455** for Flickr8K, representing 5 captions for each of the 8,091 images).


In [ ]:
captions_IDs = []
for i in range(len(cleaned_captions)):
    item = captions[i].split(',')[0]+'\t'+'start '+cleaned_captions[i]+' end\n'
    captions_IDs.append(item)

captions_IDs[:20:3], len(captions_IDs)

# Data Visualization: Images and Captions

This cell defines a function to visually inspect the dataset. It creates a side-by-side display of images and their corresponding multi-sentence descriptions to verify that the text cleaning and image mapping steps were successful.

---

## 1. Dictionary Construction

Because each image has five unique captions, the function first reorganizes the flat captions_IDs list into a more manageable structure:

* **captions_dictionary**: Groups all descriptions under a single image filename key.
* **Slicing**: The function specifically samples from index 100 to show a variety of images later in the dataset.

## 2. Image Loading and Subplot Layout

The function uses matplotlib to create a grid for visual comparison:

* **load_img**: Loads the raw image from the directory and resizes it to  pixels for consistent display.
* **Grid Logic**: It creates a subplot with num_of_images rows and 2 columns:
* **Column 1**: Displays the actual image.
* **Column 2**: An empty plot area used purely for rendering the text of the five captions.



## 3. Text Rendering

Instead of printing text to the console, the function uses the plot's coordinate system to draw the captions:

* **ax.text()**: Places each of the 5 captions at specific vertical intervals.
* **Formatting**: Sets the font size to 20 and hides the plot axes (axis('off')) to ensure the descriptions are legible and the UI is clean.

---

## 4. Execution Logic

* **visualaization(captions_IDs, 5)**: This call generates a large figure showing 5 distinct images and their 25 total captions (5 per image). This is a vital qualitative check to ensure the "start" and "end" tokens are correctly placed and the image-caption alignment is accurate.

In [ ]:
def visualaization(data, num_of_images):
    captions_dictionary = {}
    for item in data[100:100+(num_of_images)*5]:
        image_id, caption = item.split('\t')
        if image_id not in captions_dictionary:
            captions_dictionary[image_id] = []
        captions_dictionary[image_id].append(caption)
    else:
        list_captions = [x for x in captions_dictionary.items()]

    count = 1
    fig = plt.figure(figsize=(10,20))
    for filename in list(captions_dictionary.keys()):
        captions = captions_dictionary[filename]
        image_load = load_img(images_directory+filename, target_size=(199,199,3))

        ax = fig.add_subplot(num_of_images,2,count,xticks=[],yticks=[])
        ax.imshow(image_load)
        count += 1

        ax = fig.add_subplot(num_of_images,2,count)
        plt.axis('off')
        ax.plot()
        ax.set_xlim(0,1)
        ax.set_ylim(0,len(captions))
        for i, caption in enumerate(captions):
            ax.text(0,i,caption,fontsize=20)
        count += 1
    plt.show()

visualaization(captions_IDs, 5)

# Exploratory Data Analysis: Caption Length Distribution

This cell performs a statistical analysis of the caption lengths. Understanding the distribution of word counts is a prerequisite for determining the optimal sequence length for the neural network's input layers.

---

## 1. Sequence Length Analysis

The function captions_length processes the cleaned_captions list to visualize the frequency of different sentence lengths:

* **Word Counting**: It calculates the length of each caption using a list comprehension: [len(x.split(' ')) for x in data].
* **Histogram and KDE**:
* **Histogram**: Displays the count (frequency) of captions for each specific word count.
* **KDE (Kernel Density Estimate)**: Overlays a smooth curve to show the underlying probability density, helping to identify the "typical" caption length.



## 2. Importance for Model Configuration

This visualization is critical for setting the **max_length** parameter in the preprocessing pipeline:

* **Padding Strategy**: Most deep learning models require fixed-size input vectors. By looking at this plot, we can choose a maximum length that covers the vast majority of captions without being excessively long (which would lead to sparse data and wasted computation).
* **Outlier Detection**: Identifies if there are unusually long captions that might need to be truncated or removed.

---

## 3. Visualization Parameters

* **binwidth=1**: Ensures that each bar in the histogram represents exactly one word increment.
* **Styling**: Uses darkgrid and high-resolution settings (dpi=300) to ensure the plot is clear and professional.
* **Axes**:
* **X-axis (Length)**: Represents the total number of words in a caption.
* **Y-axis (Frequency)**: Represents how many captions in the dataset have that specific length.



---

## 4. Expected Insights

Typically, for the Flickr8K dataset, the distribution is skewed:

* Most captions are between  and  words long.
* Very few captions exceed  or  words.
* Based on this plot, a max_length of roughly  to  is usually selected to ensure no information is lost.


In [ ]:
def captions_length(data):
    plt.figure(figsize=(15, 7), dpi=300)
    sns.set_style('darkgrid')
    sns.histplot(x=[len(x.split(' ')) for x in data], kde=True, binwidth=1)
    plt.title('Captions length histogram', fontsize=15, fontweight='bold')
    plt.xticks(fontweight='bold')
    plt.yticks(fontweight='bold')
    plt.xlabel('Length', fontweight='bold')
    plt.ylabel('Freaquency', fontweight='bold')
    plt.show()

captions_length(cleaned_captions)

# Vocabulary Initialization and Tokenization

This cell executes the tokenization process for the entire corpus and calculates the final vocabulary size. This is a foundational step for the **Embedding layer** of the neural network.

---

## 1. Tokenizer Execution

The code calls the previously defined tokenize_captions function on the cleaned_captions list.

* **Internal Mapping**: The Tokenizer scans every word in the dataset and creates a unique integer index for each one (e.g., "dog"  1, "cat"  2).
* **Word Indexing**: It builds a dictionary accessible via tokenizer.word_index, which maps every unique word to its corresponding numeric ID.

## 2. Vocabulary Size Calculation

The variable vocab_size represents the total number of unique words the model will be able to recognize and generate.

* **len(tokenizer.word_index)**: Counts the unique words found in the cleaned captions.
* **The + 1 Addition**: In Keras, index 0 is reserved for **padding** (used to make sequences the same length). Therefore, the total vocabulary size must be the number of words plus one to account for this reserved index.

---

## 3. Structural Significance

The vocab_size is a critical hyperparameter for the model architecture:

* **Embedding Layer**: Defines the input dimension of the embedding table.
* **Output Layer**: Determines the number of neurons in the final Dense layer (using Softmax) to predict the next word in a sequence.

---

## 4. Output

* **vocab_size**: Printing this value reveals the total complexity of the language the model needs to learn. For the Flickr8K dataset, this is typically around **8,000 to 9,000** unique tokens depending on the cleaning steps used.

In [ ]:
tokenizer = tokenize_captions(cleaned_captions)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

# Dataset Splitting and Distribution

This cell implements the data partitioning strategy, ensuring that the model is trained, validated, and tested on mutually exclusive sets of images. This prevents data leakage and allows for an unbiased evaluation of the model's performance.

---

## 1. Directory Listing

* **all_image_ids**: Retrieves the filenames for all 8,091 images in the Flickr8K dataset. This list serves as the primary index for the splitting logic.

## 2. Three-Way Data Split

The code utilizes train_test_split twice to divide the image IDs into three distinct categories:

* **Train ()**: Used to update model weights and learn the relationship between images and text.
* **Validation ()**: Used during the training process to monitor for overfitting and tune hyperparameters.
* **Test ()**: Reserved for the final evaluation to see how the model generalizes to completely unseen images.

## 3. Caption Categorization

Since each image is associated with 5 captions, the code iterates through captions_IDs to ensure that all 5 captions for a specific image stay within the same split:

* **Lookup Logic**: It extracts the image_id from the tab-separated string and checks which ID set (Train, Val, or Test) it belongs to.
* **Consistency**: This approach ensures that if an image is in the test set, the model has never seen *any* of its descriptions during training.

---

## 4. Summary and Verification

The final line provides a diagnostic summary of the split:

* **Sample View**: Displays the first caption from each split to verify the format.
* **Image Counts**: The len(...) / 5 calculation converts the number of individual captions back into the number of unique images per set.

| Split | Percentage | Purpose |
| --- | --- | --- |
| **Train** |  | Model learning and weight optimization. |
| **Validation** |  | Tuning and early stopping. |
| **Test** |  | Final performance benchmarking. |

In [ ]:
all_image_ids = os.listdir(images_directory)

train_image_ids, val_image_ids = train_test_split(all_image_ids, test_size=0.15, random_state=42)
val_image_ids, test_image_ids = train_test_split(val_image_ids, test_size=0.1, random_state=42)

train_captions, val_captions, test_captions = [], [], []
for caption in captions_IDs:
    image_id, _ = caption.split('\t')

    if image_id in train_image_ids:
        train_captions.append(caption)

    elif image_id in val_image_ids:
        val_captions.append(caption)

    elif image_id in test_image_ids:
        test_captions.append(caption)

    else:
        print('Unknown image ID !')

train_captions[0], val_captions[0], test_captions[0], len(train_captions)/5, len(val_captions)/5, len(test_captions)/5

# Image Preprocessing and Feature Extraction Model

This cell prepares the **Encoder** portion of the Image Captioning system. It uses **Transfer Learning** to leverage a powerful pre-trained Convolutional Neural Network (CNN) for identifying visual patterns.

---

## 1. Image Preprocessing Pipeline (preprocess_image)

Before an image can be processed by a CNN, it must be transformed into a specific mathematical format:

* **Resizing**: load_img(..., target_size=(299, 299)) scales images to the exact dimensions required by the InceptionV3 architecture.
* **Array Conversion**: img_to_array converts the visual image into a 3D NumPy array of pixel values.
* **Dimensional Expansion**: expand_dims(..., axis=0) adds a "batch dimension," turning the  array into a  tensor.
* **Normalization**: preprocess_input scales pixel values (typically between -1 and 1) to match the distribution of the original ImageNet training data.

## 2. Feature Extraction (extract_image_features)

This function passes the preprocessed image through the model to obtain a "bottleneck" feature vector.

* **model.predict**: Runs the forward pass of the neural network.
* **verbose=0**: Suppresses the prediction progress bar for a cleaner output during batch processing.

## 3. InceptionV3 Model Modification

The script downloads the **InceptionV3** model but modifies its architecture for feature extraction rather than classification:

* **Weights ('imagenet')**: Loads the model pre-trained on millions of images from the ImageNet database.
* **Removing the Head**: The final classification layer (which predicts object categories like "dog" or "car") is removed using pop().
* **The Functional API**: Model(inputs=..., outputs=...) creates a new model that outputs the contents of the **Global Average Pooling** layer (or the second-to-last layer).

---

## 4. Architectural Goal

By truncating the model, we treat the CNN as a "vision-to-math" converter. Instead of getting a class label, we receive a high-dimensional vector (usually of size **2048**) that represents the semantic content of the image.

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(299, 299))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img

def extract_image_features(model, image_path):
    img = preprocess_image(image_path)
    features = model.predict(img, verbose=0)
    return features

inception_v3_model = InceptionV3(weights = 'imagenet', input_shape=(299, 299, 3))
inception_v3_model.layers.pop()
inception_v3_model = Model(inputs=inception_v3_model.inputs, outputs=inception_v3_model.layers[-2].output)

# Batch Feature Extraction and Persistence

This cell performs the computationally intensive task of passing every image in the dataset through the modified InceptionV3 model to extract their visual signatures.

---

## 1. Feature Storage Structure

The code initializes three dictionaries: train_image_features, val_image_features, and test_image_features.

* **Key**: The unique Image ID (filename).
* **Value**: A high-dimensional numerical vector representing the image's content.
* **Flattening**: image_features.flatten() converts the multidimensional output from the CNN into a 1D array (typically of size **2048**), making it compatible with the subsequent dense layers of the captioning model.

## 2. Progress Tracking (tqdm)

Because processing over 8,000 images through a deep neural network is time-consuming, a progress bar is implemented:

* **total=len(all_image_ids)**: Sets the bar's length to the total number of images.
* **colour='green'**: Provides a visual cue in the notebook environment to monitor the extraction status in real-time.

## 3. Extraction Logic

The loop iterates through every image ID and performs the following:

1. **Path Construction**: Combines the directory path with the image ID to locate the file.
2. **Inference**: Calls extract_image_features to perform a forward pass through InceptionV3.
3. **Conditional Sorting**: Assigns the resulting vector to the appropriate dictionary based on whether the image belongs to the **Train**, **Validation**, or **Test** split.

---

## 4. Technical Impact

By extracting and storing these features now, we avoid running the heavy InceptionV3 model during every epoch of the training phase. This **Feature Caching** strategy:

* **Saves Time**: Training will be significantly faster since the model only needs to learn the mapping from a static vector to a sequence of words.
* **Saves Memory**: We no longer need to keep the large InceptionV3 model in GPU memory once this loop completes.

---

### Expected Data Shape

| Component | Representation | Size (Typical) |
| --- | --- | --- |
| **Input** | Raw Image () | 268,203 values |
| **Output** | Flattened Feature Vector | 2,048 values |

In [ ]:
train_image_features, val_image_features, test_image_features = {}, {}, {}  # A Dictionary to store image features with their corresponding IDs

pbar = tqdm_notebook(total=len(all_image_ids), position=0, leave=True, colour='green')

for caption in all_image_ids:
    image_id = caption.split('\t')[0]
    image_path = os.path.join(images_directory, image_id)
    image_features = extract_image_features(inception_v3_model, image_path) # Extracting features

    if image_id in train_image_ids:
        train_image_features[image_id] = image_features.flatten()  # Flattening the features
        pbar.update(1)

    elif image_id in val_image_ids:
        val_image_features[image_id] = image_features.flatten()  # Flattening the features
        pbar.update(1)

    elif image_id in test_image_ids:
        test_image_features[image_id] = image_features.flatten()  # Flattening the features
        pbar.update(1)

    else:
        print('Unknown image ID !')

pbar.close()

# Supervised Learning Data Generator

This cell implements the **Data Generator** pattern, which is essential for training deep learning models on large datasets. Instead of loading the entire processed dataset into RAM (which would cause a memory overflow), this generator creates training samples **on-the-fly**.

---

## 1. The Generator Logic (data_generator)

The function follows a "Progressive Step" approach to teach the model how to predict the next word in a sequence:

* **Shuffling**: np.random.shuffle(image_ids) ensures the model doesn't learn the order of the images, improving generalization.
* **Sequence Slicing**: For a caption like start a dog runs end, the generator creates multiple training pairs:
* **Input:** start  **Target:** a
* **Input:** start a  **Target:** dog
* **Input:** start a dog  **Target:** runs


* **Padding**: pad_sequences ensures all input text vectors have a uniform length (max_caption_length), filling empty slots with zeros.
* **One-Hot Encoding**: to_categorical converts the target word index into a sparse vector of the size of the vocabulary.

## 2. Parameter Configuration

* **max_caption_length**: Dynamically calculated based on the longest caption in the dataset. Adding + 1 ensures there is room for the "start/end" tokens.
* **cnn_output_dim**: Captured as **2048**, matching the output of the InceptionV3 bottleneck layer.
* **batch_size**: Set to **270** for training and **150** for validation. This balances computational speed with memory stability.

---

## 3. Data Flow Summary

The generator yields a dual-input structure required by the **Merge Architecture** model:

| Input Component | Description | Shape |
| --- | --- | --- |
| **X_images** | Pre-extracted CNN features. | (Batch, 2048) |
| **X_captions** | Padded integer sequences of words. | (Batch, max_length) |
| **y (Target)** | One-hot encoded "next word" in sequence. | (Batch, vocab_size) |

---

## 4. Operational Setup

* **train_data_generator**: Supplies the training loop with infinite batches of randomized image-text pairs.
* **val_data_generator**: Supplies the validation loop to monitor performance on unseen data during training.

In [ ]:
def data_generator(captions, image_features, tokenizer, max_caption_length, batch_size):
    num_samples = len(captions)
    image_ids = list(image_features.keys())
    while True:
        np.random.shuffle(image_ids)  # Shuffle image_ids for each epoch
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            X_images, X_captions, y = [], [], []
            for caption in captions[start_idx:end_idx]:
                image_id, caption_text = caption.split('\t')
                caption_text = caption_text.rstrip('\n')
                seq = tokenizer.texts_to_sequences([caption_text])[0] # Tokenizing the caption
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i] # X_caption, Y
                    in_seq = pad_sequences([in_seq], maxlen=max_caption_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    X_images.append(image_features[image_id])
                    X_captions.append(in_seq)
                    y.append(out_seq)

            yield [np.array(X_images), np.array(X_captions)], np.array(y)


max_caption_length = max(len(caption.split()) for caption in cleaned_captions) + 1

cnn_output_dim = inception_v3_model.output_shape[1] # 2048

batch_size_train = 270
batch_size_val = 150

train_data_generator = data_generator(train_captions, train_image_features, tokenizer, max_caption_length, batch_size_train)
val_data_generator = data_generator(val_captions, val_image_features, tokenizer, max_caption_length, batch_size_val)

# Model Architecture: The Merge Architecture

This cell defines and compiles the core neural network. The design follows the **Merge Architecture**, which treats the image features and the text sequences as two separate encoded inputs that are combined later in the pipeline to predict the next word.

---

## 1. The Encoder Sub-networks

The model processes two distinct types of data simultaneously:

### Image Feature Branch (Input 1)

* **Input_image**: Receives the 2048-dimensional vector from the InceptionV3 model.
* **BatchNormalization**: Standardizes the features to speed up training and provide regularization.
* **Dense(256)**: Projects the high-dimensional image features into a 256-dimensional space to match the size of the language model's output.

### Sequence Processing Branch (Input 2)

* **Input_caption**: Receives the padded integer sequences of captions.
* **Embedding**: Converts integer word IDs into dense vectors of size 256. The mask_zero=True parameter tells the model to ignore the padding zeros.
* **LSTM(256)**: A Long Short-Term Memory layer that captures the temporal dependencies and context of the words in the caption.

---

## 2. The Decoder (The Merge Layer)

* **add([fe3, se2])**: This is the "Merge" step. It element-wise adds the processed image vector and the LSTM output, effectively injecting the visual context into the linguistic sequence.
* **Dense(256)**: A final hidden layer to learn the non-linear relationship between the merged features.
* **Output_Layer**: A Dense layer with a **Softmax** activation and a size equal to vocab_size. It outputs a probability distribution across all possible words in the vocabulary for the "next" word in the sequence.

---

## 3. Training Configuration

* **Loss Function**: categorical_crossentropy is used because the task is a multi-class classification problem (predicting one word out of thousands).
* **Optimizer**: **Adam** with a learning rate of 0.01.
* **clipnorm=1.0**: Implements gradient clipping to prevent "exploding gradients," a common issue in LSTM training where gradients become too large and cause instability.

---

## 4. Model Summary

* **caption_model.summary()**: Displays the layer-by-layer breakdown, showing the number of trainable parameters. This is useful for verifying that the shapes of the image and text branches align correctly before starting the training process.

In [ ]:
def build_model(vocab_size, max_caption_length, cnn_output_dim):
    input_image = Input(shape=(cnn_output_dim,), name='Features_Input')
    fe1 = BatchNormalization()(input_image)
    fe2 = Dense(256, activation='relu')(fe1) # Adding a Dense layer to the CNN output to match the decoder output size
    fe3 = BatchNormalization()(fe2)

    input_caption = Input(shape=(max_caption_length,), name='Sequence_Input')
    se1 = Embedding(vocab_size, 256, mask_zero=True)(input_caption)
    se2 = LSTM(256)(se1)

    decoder1 = add([fe3, se2])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax', name='Output_Layer')(decoder2)

    model = Model(inputs=[input_image, input_caption], outputs=outputs, name='Image_Captioning')
    return model

caption_model = build_model(vocab_size, max_caption_length, cnn_output_dim)

optimizer = Adam(learning_rate=0.01, clipnorm=1.0)
caption_model.compile(loss='categorical_crossentropy', optimizer=optimizer)

caption_model.summary()

# Model Visualization and Documentation

This cell generates a graphical representation of the neural network's architecture, providing a visual map of how data flows from the image and text inputs to the final prediction.

---

The plot_model function creates a flowchart of the model's layers. This is essential for verifying the **Dual-Input Merge Architecture**:

* **Parallel Inputs**: You can see the Features_Input (Image) and Sequence_Input (Text) branches starting independently.
* **Feature Processing**: Shows the sequence of Batch Normalization, Dense, and LSTM layers.
* **The Merge Point**: Highlights the add layer where visual and textual information are fused.
* **Prediction Head**: Shows the final transition to the vocab_size output.

In [ ]:
plot_model(caption_model)

# Final Data Pipeline and Model Training

This cell finalizes the training setup by addressing hardware compatibility, data streaming efficiency, and optimization strategies. It converts the raw Python generator into a high-performance **TensorFlow Dataset** object.

---

## 1. Optimized Data Generator (final_data_generator)

This refined version of the generator includes critical fixes for stability:

* **CuDNN Compatibility**: padding='post' is used in pad_sequences. This ensures that zeros are added at the end of the sequence rather than the beginning, which is a requirement for certain GPU-accelerated LSTM kernels (CuDNN).
* **Robust Parsing**: Includes logic to handle both tab-separated and comma-separated caption files, preventing runtime errors during data ingestion.
* **Feature Validation**: Added a check (if image_id not in image_features) to skip missing data points, ensuring the training loop doesn't crash if an image failed feature extraction.

## 2. TensorFlow Dataset Integration

To maximize GPU utilization, the generator is wrapped in tf.data.Dataset:

* **Output Signature**: Defines the exact TensorSpec (shapes and data types) for the image vectors, caption sequences, and target words. This allows TensorFlow to pre-allocate memory and optimize data flow.
* **Lazy Loading**: Data is generated only when needed, keeping the RAM footprint low.

## 3. Training Callbacks and Optimization

The training process is governed by two key automated behaviors:

* **Early Stopping**: Monitors val_loss and stops training if the model fails to improve for 3 consecutive epochs (patience=3). It also restores the best performing weights.
* **Learning Rate Decay**: The lr_scheduler reduces the learning rate exponentially as epochs progress. This allows the model to make large updates early on and smaller, more precise adjustments as it converges.

---

## 4. Execution Parameters

The model.fit call kicks off the actual learning process:

* **Epochs**: Set to 15, providing enough time for the LSTM to learn language patterns.
* **Steps per Epoch**: Calculated as Total Captions / Batch Size, ensuring the model sees every sample exactly once per epoch.

| Parameter | Value | Purpose |
| --- | --- | --- |
| **Padding** | post | GPU/CuDNN Hardware Acceleration. |
| **Max Epochs** | 15 | Total training iterations. |
| **Learning Rate** | Dynamic | Prevents overshooting the global minimum. |
| **Validation** | Enabled | Real-time monitoring of generalization. |

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# --- 1. Redefine Generator with 'padding=post' (Fixes GPU/CuDNN Error) ---
def final_data_generator(captions, image_features, tokenizer, max_len, batch_size, vocab_size):
    num_samples = len(captions)
    while True:
        # Shuffle to help training
        np.random.shuffle(captions)

        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            X_images, X_captions, y = [], [], []

            for caption_line in captions[start_idx:end_idx]:
                # Handle both tab (original) and comma (your file) splits
                if '\t' in caption_line:
                    image_id, caption_text = caption_line.strip().split('\t', 1)
                else:
                    parts = caption_line.strip().split(',')
                    image_id = parts[0]
                    caption_text = " ".join(parts[1:])

                if image_id not in image_features:
                    continue

                # Tokenize
                seq = tokenizer.texts_to_sequences([caption_text])[0]

                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]

                    # CRITICAL FIX: padding='post' puts zeros at the end.
                    # This satisfies the GPU/CuDNN requirement.
                    in_seq = pad_sequences([in_seq], maxlen=max_len, padding='post')[0]

                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                    X_images.append(image_features[image_id])
                    X_captions.append(in_seq)
                    y.append(out_seq)

            # Yield as TUPLE (Image, Caption), Target
            if len(X_images) > 0:
                yield (np.array(X_images), np.array(X_captions)), np.array(y)

# --- 2. Define Output Signature (Fixes TypeError) ---
output_signature = (
    (
        tf.TensorSpec(shape=(None, 2048), dtype=tf.float32),
        tf.TensorSpec(shape=(None, max_caption_length), dtype=tf.int32)
    ),
    tf.TensorSpec(shape=(None, vocab_size), dtype=tf.float32)
)

# --- 3. Create Datasets ---
train_dataset = tf.data.Dataset.from_generator(
    lambda: final_data_generator(train_captions, train_image_features, tokenizer, max_caption_length, batch_size_train, vocab_size),
    output_signature=output_signature
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: final_data_generator(val_captions, val_image_features, tokenizer, max_caption_length, batch_size_val, vocab_size),
    output_signature=output_signature
)

# --- 4. Callbacks (Fixes Float ValueError) ---
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def lr_scheduler(epoch, lr):
    return float(lr * tf.math.exp(-0.6))

lr_schedule = LearningRateScheduler(lr_scheduler)

# --- 5. Train ---
print("Starting training with corrected padding...")
history = caption_model.fit(
    train_dataset,
    steps_per_epoch=len(train_captions) // batch_size_train,
    validation_data=val_dataset,
    validation_steps=len(val_captions) // batch_size_val,
    epochs=15,
    callbacks=[early_stopping, lr_schedule]
)

# Model Performance Visualization

This cell generates a line graph to evaluate the model's learning progress over time. By comparing training loss against validation loss, we can diagnose the quality of the fit and determine if the model is learning effectively or simply memorizing the training data.

---

## 1. Plotting Training Dynamics

The code uses matplotlib and seaborn to visualize the history object returned by the model.fit() function:

* **Training Loss (Red):** Represents how well the model is minimizing error on the data it sees every epoch.
* **Validation Loss (Dark Maroon):** Represents the error on the unseen validation set. This is the true indicator of the model's ability to generalize to new images.

## 2. Interpreting the Results

The shape of these curves provides critical insights into the training health:

* **Convergence:** If both lines trend downward and eventually flatten, the model has successfully converged.
* **Overfitting:** If the Training Loss continues to drop while the Validation Loss begins to rise, the model is overfitting. The EarlyStopping callback used in the previous step is designed to catch this and stop training at the optimal point.
* **Stability:** The markers (o and h) help identify if the loss is decreasing smoothly or fluctuating wildly, which might suggest the learning rate is too high.

---

## 3. Visualization Features

* **whitegrid Style:** Provides a clean background with subtle grid lines, making it easier to track the numerical value of the loss across epochs.
* **DPI=200:** Ensures a high-resolution export, suitable for technical reports or presentations.
* **Epoch Offset:** The code uses x+1 for the x-axis to ensure the plot starts at "Epoch 1" instead of the default 0-indexed "Epoch 0."

---

## 4. Expected Outcome

In a successful Image Captioning run, you should see a sharp initial drop in loss as the model learns basic language structure, followed by a slower decline as it begins to associate specific visual features (from the InceptionV3 vectors) with complex descriptive words.

In [ ]:
plt.figure(figsize=(15, 7), dpi=200)
sns.set_style('whitegrid')
plt.plot([x+1 for x in range(len(history.history['loss']))], history.history['loss'], color='#E74C3C', marker='o')
plt.plot([x+1 for x in range(len(history.history['loss']))], history.history['val_loss'], color='#641E16', marker='h')
plt.title('Train VS Validation', fontsize=15, fontweight='bold')
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.xlabel('Epoch', fontweight='bold')
plt.ylabel('Loss', fontweight='bold')
plt.legend(['Train Loss', 'Validation Loss'], loc='best')
plt.show()

# Inference: Greedy Search Caption Generation

This cell defines the greedy_generator function, which is the core logic used to generate text for new, unseen images. Unlike training, where we provide the correct words, this function requires the model to predict one word at a time based on its own previous choices.

---

## 1. The Greedy Search Strategy

The function follows a **Greedy Search** approach, which is the simplest form of sequence generation:

* **Initial Seed**: The process starts with the single word 'start'.
* **Iterative Prediction**: In each loop iteration, the model takes the current sequence and the image features to predict the single most likely next word.
* **Selection**: np.argmax(prediction) selects the index of the word with the highest probability score.
* **Concatenation**: The chosen word is appended to the string, and this new, longer string is fed back into the model for the next step.

## 2. Loop Control and Termination

The generation process continues until one of two conditions is met:

* **Token Match**: If the model predicts the word 'end', the loop breaks, signaling that the sentence is complete.
* **Maximum Length**: If the 'end' token is never reached, the loop terminates after max_caption_length steps to prevent infinite recursion or nonsensical run-on sentences.

## 3. Input Reshaping and Preprocessing

To match the model's expected input shape:

* **Sequence Handling**: The text is converted to integers and padded with padding='post' to match the exact training configuration.
* **Reshaping**: Both the image features and the text sequence are reshaped into batches of size 1 ((1, cnn_output_dim) and (1, max_caption_length)) because we are processing one image at a time.

---

## 4. Final Formatting

Once the loop finishes, the function performs a "cleanup" step:

* **Token Stripping**: The internal markers start and end are removed from the string.
* **Result**: It returns a clean, human-readable sentence (e.g., "a dog runs through the grass").

In [ ]:
def greedy_generator(image_features):
    in_text = 'start'
    for _ in range(max_caption_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]

        sequence = pad_sequences([sequence], maxlen=max_caption_length, padding='post').reshape((1,max_caption_length))

        prediction = caption_model.predict([image_features.reshape(1,cnn_output_dim), sequence], verbose=0)
        idx = np.argmax(prediction)
        word = tokenizer.index_word[idx]
        in_text += ' ' + word
        if word == 'end':
            break

    in_text = in_text.replace('start ', '')
    in_text = in_text.replace(' end', '')
    return in_text



# Advanced Inference: Beam Search Generator

This cell implements **Beam Search**, a sophisticated heuristic search algorithm that improves caption quality by exploring multiple potential word sequences simultaneously. Unlike Greedy Search, which only picks the single best word at each step, Beam Search reduces the risk of choosing a word that seems good locally but leads to a poor sentence overall.

---

## 1. How Beam Search Works

Beam Search expands the model's "vision" by maintaining the **Top-K** most likely partial sentences at every step:

- **Beam Width ():** The number of parallel sequences tracked (set by K_beams).
- **Candidate Expansion:** For every active sequence, the model predicts the next word and calculates the probability for all possibilities.
- **Pruning:** After generating all possible continuations, the algorithm sorts them by their cumulative probability and keeps only the top candidates.

## 2. Probability Accumulation

The function tracks the "score" of each sequence using two methods:

- **Linear Probability:** Directly adding the Softmax probabilities.
- **Log Probability (log=True):** Adding the logarithms of probabilities. This is standard in deep learning to prevent **numerical underflow**, as multiplying many small decimal probabilities can result in numbers too small for a computer to handle accurately.

---

## 3. Algorithm Refinement

The code follows a specific sequence of operations:

1. **Initialize:** Starts with the start token and a score of 0.0.
2. **Predict:** Uses np.argsort(preds[0])[-K_beams:] to find the most likely next word indices.
3. **Sort and Slice:** start_word[-K_beams:] ensures that only the best paths survive to the next iteration.
4. **Final Selection:** Once the maximum length is reached or the "beam" stabilizes, the path with the highest cumulative score is chosen as the final caption.

---

## 4. Post-Processing

- **Index-to-Word Conversion:** Translates the winning integer sequence back into human-readable strings using the tokenizer.
- **Clean Termination:** Iterates through the list and stops at the 'end' token, ensuring the final string doesn't include the internal markers or unnecessary padding.

### Comparison: Greedy vs. Beam Search

| Feature         | Greedy Search        | Beam Search                          |
| --------------- | -------------------- | ------------------------------------ |
| **Complexity**  | Low (Fast)           | Higher (Slower)                      |
| **Exploration** | Single Path          | Multiple Paths ()                    |
| **Optimal?**    | Rarely               | More likely to find better sentences |
| **Risk**        | High (Short-sighted) | Low (Context-aware)                  |


In [ ]:
def beam_search_generator(image_features, K_beams=3, log=False):
    start = [tokenizer.word_index['start']]
    start_word = [[start, 0.0]]

    for _ in range(max_caption_length):
        temp = []
        for s in start_word:
            sequence = pad_sequences([s[0]], maxlen=max_caption_length, padding='post').reshape((1,max_caption_length))

            preds = caption_model.predict([image_features.reshape(1,cnn_output_dim), sequence], verbose=0)
            word_preds = np.argsort(preds[0])[-K_beams:]

            for w in word_preds:
                next_cap, prob = s[0][:], s[1]
                next_cap.append(w)
                if log:
                    prob += np.log(preds[0][w])
                else:
                    prob += preds[0][w]
                temp.append([next_cap, prob])

        start_word = temp
        start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
        start_word = start_word[-K_beams:]

    start_word = start_word[-1][0]
    captions_ = [tokenizer.index_word[i] for i in start_word]

    final_caption = []
    for i in captions_:
        if i != 'end':
            final_caption.append(i)
        else:
            break

    final_caption = ' '.join(final_caption[1:])
    return final_caption

# Model Evaluation: BLEU Score Analysis

This cell defines the quantitative evaluation logic for the model. It uses the **BLEU (Bilingual Evaluation Understudy)** score, which is the standard metric in Natural Language Processing for comparing a machine-generated sentence against one or more human-written reference sentences.

---

## 1. BLEU Score Fundamentals

The function measures the "precision" of the model by checking how many words (unigrams) and sequences of words (n-grams) in the predicted caption also appear in the human references:

* **BLEU-1 (Unigrams):** Focuses on individual word accuracy (vocabulary coverage).
* **BLEU-2/4 (N-grams):** Focuses on word order and fluency by checking sequences of 2 or more words.

## 2. Smoothing Function

The code implements SmoothingFunction().method1 (Chen & Cherry). This is a critical adjustment for short sequences like image captions.

* **The Problem:** Standard BLEU scores drop to  if even a single n-gram (like a 4-gram) is missing.
* **The Solution:** Smoothing provides a tiny positive value to missing n-grams so that the overall score remains representative of the words that *did* match.

## 3. Weighted Evaluation

The function uses specific weights to balance the importance of different n-gram lengths:

* **weights_1 (0.3, 0.3, 0.3, 0):** Evaluates matches up to 3-grams.
* **weights_2 (0.25, 0.25, 0.25, 0.25):** Evaluates matches up to 4-grams (standard BLEU-4 configuration).

---

## 4. Comparison Logic

The function evaluates both generation strategies simultaneously:

1. **Greedy Scoring:** Calculates how accurate the most probable word-by-word path was.
2. **Beam Search Scoring:** Calculates if the multi-path exploration resulted in a higher-quality, more "human-like" caption.

---

## 5. Output Format

The function returns a formatted list of strings for easy readability:

* **Quantitative Data:** Rounded BLEU-1 and BLEU-2 scores for both methods.
* **Qualitative Data:** The actual generated strings for a side-by-side comparison.

| Metric | Higher Score Means... |
| --- | --- |
| **BLEU-1** | The model is using the correct objects/labels. |
| **BLEU-2+** | The model is constructing grammatically correct phrases. |



In [ ]:
def BLEU_score(actual, greedy, beam_search):
    # 1. Setup: Tokenize & Lowercase
    # "actual" is a list of reference strings. We split them into words.
    references = [s.lower().split() for s in actual]

    # "greedy" and "beam_search" are lists containing one prediction string.
    greedy_tokens = greedy[0].lower().split()
    beam_tokens = beam_search[0].lower().split()

    # 2. Define Smoothing Function
    # This prevents the score from crashing to 0.0 just because a 4-gram is missing.
    chen_cherry = SmoothingFunction()

    # "BLEU-1" in your tutorial uses (0.3, 0.3, 0.3, 0) -> Looks for up to 3-grams
    weights_1 = (0.3, 0.3, 0.3, 0)
    # "BLEU-2" in your tutorial uses (0.25, 0.25, 0.25, 0.25) -> Looks for up to 4-grams
    weights_2 = (0.25, 0.25, 0.25, 0.25)

    # 4. Calculate Scores with Smoothing
    # Greedy
    sg_1 = sentence_bleu(references, greedy_tokens, weights=weights_1, smoothing_function=chen_cherry.method1)
    sg_2 = sentence_bleu(references, greedy_tokens, weights=weights_2, smoothing_function=chen_cherry.method1)

    # Beam Search
    sb_1 = sentence_bleu(references, beam_tokens, weights=weights_1, smoothing_function=chen_cherry.method1)
    sb_2 = sentence_bleu(references, beam_tokens, weights=weights_2, smoothing_function=chen_cherry.method1)

    # 5. Return in the EXACT format you requested
    return [
        f'BLEU-2 Greedy: {round(sg_2, 5)}',
        f'BLEU-1 Greedy: {round(sg_1, 5)}',
        f'Greedy: {greedy[0]}',
        f'BLEU-2 Beam Search: {round(sb_2, 5)}',
        f'BLEU-1 Beam Search: {round(sb_1, 5)}',
        f'Beam Search:  {beam_search[0]}'
    ]

# Batch Prediction on Test Data

This cell executes the inference phase for the entire test set. It uses the trained model to generate captions for images it has never seen before, allowing for a final evaluation of the system's performance.

---

## 1. Automated Caption Generation

The loop iterates through the test_image_features dictionary to produce descriptions for each held-out image:

* **greedy_generator**: The function predicts the sequence of words word-by-word, choosing the most likely token at each step based on the image features.
* **Storage**: Results are saved in the generated_captions dictionary, mapped by image_id. This structured storage makes it easy to compare the AI's output against the ground-truth human captions later.

## 2. Progress Monitoring

Since generating text for hundreds of images involves thousands of individual model predictions, the tqdm_notebook progress bar is utilized:

* **Real-time Feedback**: Tracks the completion percentage and estimated time remaining.
* **Visual Cue**: The green progress bar confirms that the inference loop is running smoothly without errors.

---

## 3. Workflow Context

This step serves as the bridge between **Model Training** and **Final Evaluation**:

1. **Extraction**: (Completed) Image features were extracted using InceptionV3.
2. **Training**: (Completed) The LSTM learned to map features to words.
3. **Inference**: (**This Step**) The model "imagines" captions for new visual data.
4. **Scoring**: (Next Step) The generated text will be compared to human references using BLEU scores.

---

## 4. Key Metrics to Watch

After this loop finishes, the generated_captions dictionary will contain the raw material needed to calculate:

* **Linguistic Diversity**: How many unique words did the model use across the test set?
* **Average Length**: Are the generated captions consistently shorter or longer than the training data?
* **Semantic Accuracy**: Do the words actually match the objects present in the images?

In [ ]:
generated_captions = {} 

pbar = tqdm_notebook(total=len(test_image_features), position=0, leave=True, colour='green')
for image_id in test_image_features:
    cap = greedy_generator(test_image_features[image_id])
    generated_captions[image_id] = cap
    pbar.update(1)

pbar.close()

# Data Alignment: Ground Truth Preparation

This cell creates a dedicated mapping of original, human-written descriptions for the test dataset. This "Ground Truth" is essential for the final evaluation phase, where we compare the model's AI-generated text against the actual descriptions provided by humans.

---

## 1. Ground Truth Dictionary

The code constructs the test_actual_captions dictionary using a logic similar to the training data preparation, but specifically filtered for the **Test Set**:

* **Key**: The unique Image ID (e.g., 12345.jpg).
* **Value**: A list containing all five human-provided captions for that specific image.
* **Format Flexibility**: The code includes a conditional check to handle both tab-separated (\t) and comma-separated (,) file formats, ensuring the script is robust across different versions of the Flickr8K dataset.

## 2. Normalization for Evaluation

* **caption_text.lower()**: This is a critical step for scoring. Since the model was trained on cleaned, lowercase text, the reference captions must also be in lowercase.
* **Case Sensitivity**: Without this normalization, a match between "Dog" (Reference) and "dog" (Prediction) would be ignored by the BLEU score algorithm, leading to artificially low performance metrics.

---

## 3. The Comparison Framework

By the end of this cell, we have two perfectly aligned structures:

1. **generated_captions**: What the model *thinks* is in the image (1 caption per image).
2. **test_actual_captions**: What humans *know* is in the image (5 captions per image).

---

## 4. Why 5 Captions per Image?

In Image Captioning, there is no single "correct" answer. One person might describe an image as "a dog running," while another says "a golden retriever in the grass." By comparing our model's output against **five** different references, we get a much fairer and more accurate BLEU score.


In [ ]:
# Create the missing dictionary 'test_actual_captions'
test_actual_captions = {}

# Iterate over your original 'captions' list
for line in captions:
    line = line.strip()
    # Handle CSV split
    if '\t' in line:
        image_id, caption_text = line.split('\t', 1)
    else:
        parts = line.split(',')
        image_id = parts[0]
        caption_text = " ".join(parts[1:])

    if image_id in test_image_ids:
        if image_id not in test_actual_captions:
            test_actual_captions[image_id] = []
        test_actual_captions[image_id].append(caption_text.lower())

# Qualitative Results: Image, Prediction, and BLEU Analysis

This final visualization function brings all components together—the raw visual data, the model's linguistic interpretation, and the mathematical validation—into a single scannable interface. It provides a "gut check" on how the model actually performs in the real world.

---

## 1. Visual Comparison Logic

The function visualization performs a comprehensive assessment of 7 randomly selected images from the test set:

* **Ground Truth Cleanup**: It strips the internal start and end tokens from the reference captions to make them human-readable.
* **Dual Inference**: It runs both the **Greedy** and **Beam Search** generators for the same image to see which one produces a more natural description.
* **Score Integration**: It calls the BLEU_score evaluator to instantly display the performance metrics alongside the text.

## 2. Layout Structure

The output is organized as a vertical gallery:

* **Left Column**: Displays the actual test image, resized for the grid.
* **Right Column**: A text-based dashboard containing:
* The **Greedy Search** prediction.
* The **Beam Search** prediction.
* **BLEU-1 and BLEU-2 scores** for both methods, providing immediate feedback on how closely the AI matched human consensus.



---

## 3. Interpreting the Outputs

When you run this cell, you can look for three specific behaviors:

1. **Semantic Alignment**: Does the model identify the correct objects (e.g., "dog", "ball", "grass")?
2. **Grammar and Fluency**: Is the sentence structure logical, or is it a "bag of words"?
3. **Beam vs. Greedy**: Does Beam Search (which looks ahead) provide a more complex or accurate description than the simple Greedy approach?

---

## 4. Summary of the Pipeline

This concludes the end-to-end Image Captioning workflow:

1. **Load/Clean**: Prepared the Flickr8K dataset.
2. **Encoder**: Extracted visual features with InceptionV3.
3. **Decoder**: Built an LSTM network to generate sequences.
4. **Training**: Optimized the model with Adam and EarlyStopping.
5. **Evaluation**: Quantified performance with BLEU and visualized results.

In [ ]:
def visualization(data, greedy_caps, beamS_generator, evaluator, num_of_images): #20
    keys = list(data.keys())
    images = [np.random.choice(keys) for i in range(num_of_images)] # Randomly selected images
    count = 1
    fig = plt.figure(figsize=(6,20))
    for filename in images:
        actual_cap = data[filename]
        actual_cap = [x.replace("start ", "") for x in actual_cap]
        actual_cap = [x.replace(" end", "") for x in actual_cap]

        greedy_cap = greedy_caps[filename]
        beamS_cap = beamS_generator(test_image_features[filename])

        caps_with_score = evaluator(actual_cap, [greedy_cap]*(len(actual_cap)), [beamS_cap]*(len(actual_cap)))

        image_load = load_img(images_directory+filename, target_size=(199,199,3))
        ax = fig.add_subplot(num_of_images,2,count,xticks=[],yticks=[])
        ax.imshow(image_load)
        count += 1

        ax = fig.add_subplot(num_of_images,2,count)
        plt.axis('off')
        ax.plot()
        ax.set_xlim(0,1)
        ax.set_ylim(0,len(caps_with_score))
        for i, text in enumerate(caps_with_score):
            ax.text(0,i,text,fontsize=10)
        count += 1
    plt.show()

visualization(test_actual_captions, generated_captions, beam_search_generator, BLEU_score, 7)